### The task:
1. Identify the annual salary for each customer
2. Explore the correlation between annual salary and various customer attribute (e.g. age)
3. Build a simple regression model to predict the annual salary for each customer 
4. Build a decision-tree based model to predict salary
5. Which model is better? 

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel("ANZ synthesised transaction dataset.xlsx")
df.shape

(12043, 23)

There are 12,0403 rows of transactions by 23 columns of data

In [3]:
df.head(2)

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit


### Task 1: Identify the annual salary for each customer

In [4]:
df["txn_description"].unique()

array(['POS', 'SALES-POS', 'PAYMENT', 'INTER BANK', 'PAY/SALARY',
       'PHONE BANK'], dtype=object)

In [5]:
# Extracting only pay or salary transactions
df_group = df[df["txn_description"]=="PAY/SALARY"].groupby("customer_id").mean()

In [6]:
# Appending the annual salary to a new column in df
salary = []

for customer_id in df["customer_id"]:
    salary.append(int(df_group.loc[customer_id]["amount"]))

df["annual_salary"] = salary

In [7]:
df_salary = df.groupby("customer_id").mean()
df_salary.head()

,card_present_flag,merchant_code,balance,age,amount,annual_salary
customer_id,,,,,,
CUS-1005756958,0.812500,0.0,2275.852055,53,222.862603,970
CUS-1117979751,0.826923,0.0,9829.929000,21,339.843700,3578
CUS-1140341822,0.815385,0.0,5699.212250,28,212.632500,1916
CUS-1147642491,0.750000,0.0,9032.841186,34,245.600169,1711
CUS-1196156254,0.785276,0.0,22272.433755,34,147.145796,3903


### Task 2: Correlation between variables

In [8]:
corr = df_salary.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,card_present_flag,merchant_code,balance,age,amount,annual_salary
card_present_flag,1,nan,0.012,-0.0013,-0.28,-0.15
merchant_code,nan,nan,nan,nan,nan,nan
balance,0.012,nan,1,0.23,0.27,0.21
age,-0.0013,nan,0.23,1,0.18,-0.14
amount,-0.28,nan,0.27,0.18,1,0.29
annual_salary,-0.15,nan,0.21,-0.14,0.29,1


It can be seen that amount and annual_salary has the highest correlation of 0.29. 
* This suggests that an increase in annual_salary will increase the amount spent

There is also a correlation between annual_salary and balance of 0.21.
* This suggest that a higher annual salary will have a higher balance

### Task 3: Regression model

#### Simple Linear Regression - Model 1

In [9]:
# X_train columns
x = df_salary.drop("annual_salary",axis=1)
y = df_salary["annual_salary"]

In [10]:
# Create training and testing vars
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 10)

In [11]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, Y_train)
# Making prediction based off model
linear_reg.predict(X_test)

array([2471.79881578, 2355.95764968, 5516.40875896, 2749.14617612,
       2463.19874194, 2210.76713385, 1628.2118905 , 2426.7591228 ,
       1832.71930055, 2092.26357951, 2122.33968281, 2750.21948899,
       2590.35003469, 2563.97503629, 1829.3763517 , 1809.30374498,
       2112.75645323, 1961.775454  , 1809.99591947, 1593.15760482])

In [12]:
r_sq = linear_reg.score(X_test, Y_test)
#print('Intercept:',linear_reg.intercept_)
#print('Slope:',linear_reg.coef_)
print("Coefficient of determination:", r_sq)

Coefficient of determination: -0.3373658380895257


The simlpe linear regression is not a suitable model as it's R(squared) value is -0.3373.

### Task 4: Decision Tree

In [13]:
df_cat = df[["txn_description", "gender", "age", "merchant_state", "movement"]]

In [14]:
pd.get_dummies(df_cat).head()

,age,txn_description_INTER BANK,txn_description_PAY/SALARY,txn_description_PAYMENT,txn_description_PHONE BANK,txn_description_POS,txn_description_SALES-POS,gender_F,gender_M,merchant_state_ACT,merchant_state_NSW,merchant_state_NT,merchant_state_QLD,merchant_state_SA,merchant_state_TAS,merchant_state_VIC,merchant_state_WA,movement_credit,movement_debit
0,26,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1
1,26,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1
2,38,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1
3,40,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1
4,26,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1


In [15]:
# X_train columns
x = pd.get_dummies(df_cat)
y = df["annual_salary"]

In [16]:
# Create training and testing vars
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 10)

#### Regression

In [19]:
decision_tree_reg = DecisionTreeRegressor()
decision_tree_reg.fit(X_train, Y_train)
decision_tree_reg.predict(X_test)

array([1327.11320755, 4405.        , 2588.        , ..., 2113.66666667,
       3071.        , 3350.98333333])

In [20]:
dt_reg = decision_tree_reg.score(X_test, Y_test)
print("Accuracy of decision tree regression model:", dt_reg)

Accuracy of decision tree regression model: 0.7191301533058805


### Task 5: Which model is better?

The decision tree regression model works better in our case. The decision tree regressor is able to achieve an accuracy of 71.91%. The simple linear regression model is a very bad model as it's R(squared) value is of a negative value. When a R(squared) value is negative, it means that the fit is actually worse than just fitting a horizontal line.